In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - CamP ZipNeRF (Jax) Gradio Notebook
<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_camp_zipnerf_gradio.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_camp_zipnerf_gradio.ipynb">
      <img alt="GitHub logo" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview
This notebook launches a Gradio application based on the [jax implementation](https://github.com/jonbarron/camp_zipnerf) of [CamP: Camera Preconditioning for Neural Radiance Fields](https://camp-nerf.github.io/). The application is designed for training and rendering Neural Radiance Fields (NeRFs) more efficiently in jax. CamP addresses some of the limitations of traditional NeRF techniques, which, while powerful for creating detailed 3D models from 2D images, can be computationally intensive and slow.

The goal of the Gradio interface is to provide a truly user-friendly experience for users with limited knowledge of Google Cloud. This ensures that anyone can easily access and leverage the powerful capabilities of CamP without needing extensive technical expertise.

## Objective

In this tutorial, you will learn how to:

- Use [COLMAP](https://colmap.github.io/) to perform Structure from Motion (SfM), a technique that estimates the three-dimensional structure of a scene from a series of two-dimensional images.
- Calibrate, train and render NERF scenes using [Vertex AI custom jobs](https://cloud.google.com/vertex-ai/docs/samples/aiplatform-create-custom-job-sample).
- Render a video along a custom camera path using a series of keyframe photos.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Training
- Vertex AI Custom Job

Additionally, we provide a comprehensive **pipeline** that automates the entire process by running all three jobs (SfM, calibration/training, and rendering) in a Directed Acyclic Graph (DAG) fashion. This pipeline ensures efficient and sequential execution of each step, streamlining the workflow and minimizing manual intervention.

## Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Setup

In [ ]:
# @title Setup Google Cloud project and prepare the dependencies

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).
# @markdown 2. [Optional] [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing
# @markdown experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`)
# @markdown should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is
# @markdown not considered a match for a single region covered by the multi-region range (eg. "us-central1").
# @markdown If not set, a unique GCS bucket will be created instead.

! pip3 install --upgrade gradio==4.29.0
! pip3 install --upgrade pandas==2.2.1
! pip3 install --upgrade opencv-python==4.10.0.84
# Uninstall nest-asyncio and uvloop as a workaround to https://github.com/gradio-app/gradio/issues/8238#issuecomment-2101066984
! pip3 uninstall --yes nest-asyncio uvloop
! pip3 install google-cloud-bigquery==3.24.0
! pip3 install kfp==2.7.0
! pip3 install google-cloud-pipeline-components==2.14.1
! pip3 install --upgrade oauth2client==1.4.2
! pip3 install six==1.16.0
! pip3 install moviepy==1.0.3


import os
from datetime import datetime

from google.cloud import aiplatform

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, please change the value yourself below.
now = datetime.now().strftime("%Y%m%d-%H%M%S")
BUCKET_URI = "gs://cloud-nas-260507-tmp-20240524-173228"  # @param {type: "string"}

assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    # Create a unique GCS bucket for this notebook, if not specified by the user
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}"
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )

print(f"Using this GCS Bucket: {BUCKET_URI}")

# Set up the default SERVICE_ACCOUNT.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
SERVICE_ACCOUNT_CC = (
    f"service-{project_number}@gcp-sa-aiplatform-cc.iam.gserviceaccount.com"
)

print("Using this default Service Account:", SERVICE_ACCOUNT)

BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
# Provision permissions to the two SERVICE_ACCOUNTs with the GCS bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin

staging_bucket = os.path.join(BUCKET_URI, "zipnerf_staging")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=staging_bucket)

PROJECT_NUMBER = project_number

# The pre-built calibration docker image.
CALIBRATION_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-cloudnerf-calibrate:latest"
# The pre-built training docker image.
TRAINING_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/jax-cloudnerf-train:latest"
# The pre-built rendering docker image.
RENDERING_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/jax-cloudnerf-render:latest"

In [2]:
# @title BigQuery Setup for CamP ZipNeRF Experiment Tracking

# @markdown The app leverages [BigQuery](https://pantheon.corp.google.com/bigquery) to create interactive dataframes that persistently tracks the lifecycle of NeRF experiments. This ensures that even if the runtime is stopped or lost, the app continues to work with the same information.

# @markdown Each user is assigned a unique database name in BigQuery, generated based on their bucket name.

from datetime import datetime

from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

PROJECT_ID = "cloud-nas-260507"  # Update this with your actual project ID
bucket_name = BUCKET_URI.replace("gs://", "").replace("-", "_")
DATASET_NAME = f"nerf_gradio_app_data_{bucket_name}"

# Define dataset and table IDs
dataset_id = f"{PROJECT_ID}.{DATASET_NAME}"
table_ids = {
    "colmap_data": f"{dataset_id}.colmap_data",
    "training_data": f"{dataset_id}.training_data",
    "rendering_data": f"{dataset_id}.rendering_data",
}

# Create dataset
dataset = bigquery.Dataset(dataset_id)
dataset = client.create_dataset(dataset, exists_ok=True)
print(f"Created app interface dataset {client.project}.{dataset.dataset_id}")
# Define schemas
schemas = {
    "colmap_data": [
        bigquery.SchemaField("Job_Status", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Pipeline_State", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Pipeline_Resource_Name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Experiment_ID", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("Colmap_Job_ID", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Scene_Name", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("Image_Count", "INT64", mode="NULLABLE"),
        bigquery.SchemaField("Created_Time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("Start_Time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("End_Time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("Matcher_Type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Camera_Type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Video_Frame_FPS", "INT64", mode="NULLABLE"),
        bigquery.SchemaField("Max_Num_Features", "INT64", mode="NULLABLE"),
        bigquery.SchemaField("Use_Hierarchical_Mapper", "BOOL", mode="NULLABLE"),
        bigquery.SchemaField("GCS_Dataset_Path", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("GCS_Experiment_Path", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Machine_Type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Accelerator_Type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Accelerator_Count", "INT64", mode="NULLABLE"),
    ],
    "training_data": [
        bigquery.SchemaField("Job_Status", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Pipeline_State", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Pipeline_Resource_Name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Experiment_ID", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("Training_Job_ID", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Training_Job_Name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Colmap_Job_ID", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Scene_Name", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("Image_Count", "INT64", mode="NULLABLE"),
        bigquery.SchemaField("Created_Time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("Start_Time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("End_Time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("Training_Factor", "INT64", mode="NULLABLE"),
        bigquery.SchemaField("Training_Max_Steps", "INT64", mode="NULLABLE"),
        bigquery.SchemaField("ZipNeRF_Gin_Config", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("CamP_Gin_Config", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("GCS_Experiment_Path", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Machine_Type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Accelerator_Type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Accelerator_Count", "INT64", mode="NULLABLE"),
    ],
    "rendering_data": [
        bigquery.SchemaField("Job_Status", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Pipeline_State", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Pipeline_Resource_Name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Experiment_ID", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("Rendering_Job_ID", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Rendering_Job_Name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Training_Job_ID", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Training_Job_Name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Colmap_Job_ID", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Scene_Name", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("Image_Count", "INT64", mode="NULLABLE"),
        bigquery.SchemaField("Created_Time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("Start_Time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("End_Time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("Render_Factor", "INT64", mode="NULLABLE"),
        bigquery.SchemaField("Render_Resolution_Width", "INT64", mode="NULLABLE"),
        bigquery.SchemaField("Render_Resolution_Height", "INT64", mode="NULLABLE"),
        bigquery.SchemaField("Render_FPS", "INT64", mode="NULLABLE"),
        bigquery.SchemaField("ZipNeRF_Gin_Config", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("CamP_Gin_Config", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("GCS_Keyframes_File", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("GCS_Render_Path_File", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Render_Camtype", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Render_Focal", "FLOAT64", mode="NULLABLE"),
        bigquery.SchemaField("Render_Path_Frames", "INT64", mode="NULLABLE"),
        bigquery.SchemaField("GCS_Experiment_Path", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Machine_Type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Accelerator_Type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Accelerator_Count", "INT64", mode="NULLABLE"),
    ],
}

# Create tables
for table_name, schema in schemas.items():
    table_id = table_ids[table_name]
    table = bigquery.Table(table_id, schema=schema)
    table = client.create_table(table, exists_ok=True)
    print(f"Created table {table.project}.{table.dataset_id}.{table.table_id}")


def insert_or_update_row(row, table_id, primary_key="Experiment_ID"):
    columns = ", ".join(row.keys())
    values = ", ".join([f"@{key}" for key in row.keys()])
    updates = ", ".join(
        [f"T.{key} = S.{key}" for key in row.keys() if key != primary_key]
    )

    query = f"""
    MERGE `{table_id}` T
    USING (SELECT {', '.join([f"{value} AS {key}" for key, value in zip(row.keys(), values.split(', '))])}) S
    ON T.{primary_key} = S.{primary_key}
    WHEN MATCHED THEN
      UPDATE SET {updates}
    WHEN NOT MATCHED THEN
      INSERT ({columns})
      VALUES ({values})
    """

    query_parameters = [
        bigquery.ScalarQueryParameter(
            key,
            (
                "STRING"
                if key
                in [
                    "Job_Status",
                    "Pipeline_State",
                    "Pipeline_Resource_Name",
                    "Experiment_ID",
                    "Colmap_Job_ID",
                    "Scene_Name",
                    "Training_Job_ID",
                    "Rendering_Job_ID",
                    "Matcher_Type",
                    "Camera_Type",
                    "ZipNeRF_Gin_Config",
                    "CamP_Gin_Config",
                    "GCS_Keyframes_File",
                    "GCS_Render_Path_File",
                    "Render_Camtype",
                    "GCS_Dataset_Path",
                    "GCS_Experiment_Path",
                    "Training_Job_Name",
                    "Rendering_Job_Name",
                    "Machine_Type",
                    "Accelerator_Type",
                ]
                else (
                    "INT64"
                    if key
                    in [
                        "Image_Count",
                        "Video_Frame_FPS",
                        "Max_Num_Features",
                        "Training_Factor",
                        "Training_Max_Steps",
                        "Render_Factor",
                        "Render_Resolution_Width",
                        "Render_Resolution_Height",
                        "Render_FPS",
                        "Render_Path_Frames",
                        "Accelerator_Count",
                    ]
                    else (
                        "BOOL"
                        if key == "Use_Hierarchical_Mapper"
                        else "FLOAT64" if key == "Render_Focal" else "TIMESTAMP"
                    )
                )
            ),
            value,
        )
        for key, value in row.items()
    ]

    job_config = bigquery.QueryJobConfig(query_parameters=query_parameters)

    query_job = client.query(query, job_config=job_config)
    query_job.result()  # Wait for the job to complete
    print(f"Inserted/Updated row with {primary_key}: {row[primary_key]}")

Created app interface dataset cloud-nas-260507.nerf_gradio_app_data_cloud_nas_260507_tmp_20240524_173228
Created table cloud-nas-260507.nerf_gradio_app_data_cloud_nas_260507_tmp_20240524_173228.colmap_data
Created table cloud-nas-260507.nerf_gradio_app_data_cloud_nas_260507_tmp_20240524_173228.training_data
Created table cloud-nas-260507.nerf_gradio_app_data_cloud_nas_260507_tmp_20240524_173228.rendering_data


In [38]:
# @title Gradio App Utility Functions


import concurrent.futures
import glob
import hashlib
import mimetypes
import os
import re
import shutil
import threading
import time
from datetime import datetime
from typing import List

import cv2
import gradio as gr
import numpy as np
import pandas as pd
from google.cloud import aiplatform, bigquery, storage
from google_cloud_pipeline_components.v1.custom_job import CustomTrainingJobOp
from google_cloud_pipeline_components.v1.vertex_notification_email import \
    VertexNotificationEmailOp
from kfp import compiler, dsl
from moviepy.editor import VideoFileClip
from PIL import Image

GCS_URI_PREFIX = "gs://"

JOB_STATE_MAPPING = {
    0: "NOT STARTED",
    1: "QUEUED",
    2: "PENDING",
    3: "RUNNING",
    4: "SUCCEEDED",
    5: "FAILED",
    6: "CANCELLING",
    7: "CANCELLED",
    8: "PAUSED",
    9: "EXPIRED",
}

PIPELINE_JOB_STATE_MAPPING = {
    "PIPELINE_STATE_UNSPECIFIED": 0,
    "PIPELINE_STATE_QUEUED": 1,
    "PIPELINE_STATE_PENDING": 2,
    "PIPELINE_STATE_RUNNING": 3,
    "PIPELINE_STATE_SUCCEEDED": 4,
    "PIPELINE_STATE_FAILED": 5,
    "PIPELINE_STATE_CANCELLING": 6,
    "PIPELINE_STATE_CANCELLED": 7,
    "PIPELINE_STATE_PAUSED": 8,
}

MATCHER_MAPPING = {
    "Exhaustive Matcher": "exhaustive",
    "Sequential Matcher": "sequential",
    "Spatial Matcher": "spatial",
    "Transitive Matcher": "transitive",
    "Vocab Tree Matcher": "vocab_tree",
}
IMAGE_EXTENSIONS = (".png", ".jpg", ".jpeg", ".gif", ".bmp")
GCS_API_ENDPOINT = "https://storage.cloud.google.com/"

unique_experiments = set()

# Define dataset and table IDs
dataset_id = f"{PROJECT_ID}.{DATASET_NAME}"
table_ids = {
    "colmap_data": f"{dataset_id}.colmap_data",
    "training_data": f"{dataset_id}.training_data",
    "rendering_data": f"{dataset_id}.rendering_data",
}

colmap_table_id = table_ids["colmap_data"]
training_table_id = table_ids["training_data"]
rendering_table_id = table_ids["rendering_data"]


def generate_short_unique_identifier():
    current_time = str(time.time()).encode("utf-8")
    hash_object = hashlib.sha256(current_time)
    short_hash = hash_object.hexdigest()[:7]
    return short_hash


def extract_dataset_id(input_string):
    # Define the regex pattern to match the dataset ID
    pattern = r"dataset_\d{8}_\d{6}"

    # Check if the input string itself matches the pattern
    if re.fullmatch(pattern, input_string):
        return input_string

    # Search for the pattern in the input string
    match = re.search(pattern, input_string)

    # If a match is found, return the matched string, otherwise return None
    if match:
        return match.group(0)
    else:
        return None


def get_job_name_with_datetime(prefix: str) -> str:
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def get_vertex_ai_job_status(job_id: str) -> str:
    job = aiplatform.CustomJob.get(job_id)
    return job.state


def get_vertex_ai_training_job_link(job_id, project_number, location="us-central1"):
    base_url = "https://console.cloud.google.com/ai/platform/locations"
    link = f"{base_url}/{location}/training/{job_id}?project={project_number}"
    return link


def get_vertex_ai_pipeline_run_link(
    pipeline_run_id, project_number, location="us-central1"
):
    base_url = "https://console.cloud.google.com/vertex-ai/locations"
    link = f"{base_url}/{location}/pipelines/runs/{pipeline_run_id}?project={project_number}"
    return link


def get_vertex_ai_pipeline_job_status(job_id: str) -> str:
    job = aiplatform.PipelineJob.get(job_id)
    return job.state


def get_bucket_and_blob_name(filepath: str) -> tuple:
    gs_suffix = filepath.split("gs://", 1)[1]
    return tuple(gs_suffix.split("/", 1))


def get_bucket_and_blob_name_https(filepath: str) -> tuple:
    gs_suffix = filepath.split("https://", 1)[1]
    return tuple(gs_suffix.split("/", 1))


def get_bigquery_client():
    return bigquery.Client()


def is_gcs_path(input_path: str) -> bool:
    """Checks if the input path is a Google Cloud Storage (GCS) path.

    Args:
        input_path: The input path to be checked.

    Returns:
        True if the input path is a GCS path, False otherwise.
    """
    return input_path is not None and input_path.startswith(GCS_URI_PREFIX)


def create_pending_video(
    pending_message="PENDING",
    output_path="pending_video.mp4",
    width=1280,
    height=720,
    duration=5,
    fps=30,
):
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Define text properties
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 5
    font_thickness = 10
    text_size = cv2.getTextSize(pending_message, font, font_scale, font_thickness)[0]

    # Calculate the center position
    text_x = (width - text_size[0]) // 2
    text_y = (height + text_size[1]) // 2

    # Create frames and write them to the video
    total_frames = duration * fps
    for _ in range(total_frames):
        frame = np.zeros((height, width, 3), dtype=np.uint8)
        cv2.putText(
            frame,
            pending_message,
            (text_x, text_y),
            font,
            font_scale,
            (255, 255, 255),
            font_thickness,
            lineType=cv2.LINE_AA,
        )
        out.write(frame)

    # Release everything if job is finished
    out.release()
    print(f"Created pending video at {output_path}")
    return output_path


def list_mp4_files(bucket_name, folder_path, job_status):
    client = storage.Client()

    # Get all the blobs (files) with .mp4 extension in the specified folder
    blobs = client.list_blobs(bucket_name, prefix=folder_path)

    mp4_files = []
    try:
        for blob in blobs:
            if blob.name.endswith(".mp4"):
                mp4_files.append(blob.name)
            else:
                mp4_files.append(create_pending_video(job_status, "pending_video.mp4"))
    except IndexError:
        mp4_files.append(create_pending_video(job_status, "pending_video.mp4"))

    return mp4_files


def list_folders(bucket_name, folder_path):
    client = storage.Client()

    # Get all the blobs (files and folders) in the specified folder
    blobs = client.list_blobs(bucket_name, prefix=folder_path)

    folders = set()
    for blob in blobs:
        # Extract the folder path from each blob's name
        folder = blob.name.rsplit("/", 1)[0]
        folders.add(folder)

    return list(folders)


def download_gcs_file_to_local_dir(gcs_uri: str, local_dir: str):
    """Download a gcs file to a local dir.

    Args:
      gcs_uri: A string of file path on GCS.
      local_dir: A string of local directory.
    """
    if not is_gcs_path(gcs_uri):
        raise ValueError(f"{gcs_uri} is not a GCS path starting with {GCS_URI_PREFIX}.")
    filename = os.path.basename(gcs_uri)
    download_gcs_file_to_local(gcs_uri, os.path.join(local_dir, filename))


def download_gcs_file_to_local(gcs_uri: str, local_path: str):
    """Download a gcs file to a local path.

    Args:
      gcs_uri: A string of file path on GCS.
      local_path: A string of local file path.
    """
    if not is_gcs_path(gcs_uri):
        raise ValueError(f"{gcs_uri} is not a GCS path starting with {GCS_URI_PREFIX}.")
    client = storage.Client()
    os.makedirs(os.path.dirname(local_path), exist_ok=True)
    with open(local_path, "wb") as f:
        client.download_blob_to_file(gcs_uri, f)


def list_gcs_bucket_contents() -> dict:
    client = storage.Client()
    bucket_name = get_bucket_and_blob_name(BUCKET_NAME)[0]
    bucket = client.get_bucket(bucket_name)
    blobs = bucket.list_blobs()
    folder_counts = {}

    for blob in blobs:
        folder = blob.name.split("/")[0]
        folder_counts[folder] = folder_counts.get(folder, 0) + 1

    return folder_counts


def upload_local_dir_to_gcs(
    scene_name: str,
    local_dir_path: str,
    gcs_dir_path: str,
    progress=gr.Progress(),
    table_id: str = "",
):
    total_files = len(glob.glob(local_dir_path + "/**"))
    completed_files = 0

    bucket_name = get_bucket_and_blob_name(BUCKET_NAME)[0]
    scene_name = scene_name + "_" + generate_short_unique_identifier()
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    dataset_folder_name = os.path.basename(gcs_dir_path)

    for local_file in glob.glob(local_dir_path + "/**"):
        if os.path.isfile(local_file):
            filename = local_file[1 + len(local_dir_path) :]
            gcs_file_path = os.path.join(gcs_dir_path, filename)
            _, blob_name = get_bucket_and_blob_name_https(gcs_file_path)
            blob = bucket.blob(blob_name)
            blob.upload_from_filename(local_file)
            completed_files += 1
            progress(completed_files / total_files)
            print(f"Copied {local_file} to {gcs_file_path}.")

    folder_counts = list_gcs_bucket_contents()
    gcs_dataset_path = os.path.join(BUCKET_NAME, dataset_folder_name)
    gcs_experiment_path = gcs_dataset_path.replace("dataset", "experiment")
    row_data = {
        "Experiment_ID": dataset_folder_name,
        "Scene_Name": scene_name,
        "Job_Status": "Images Uploaded",
        "Colmap_Job_ID": "",
        "Image_Count": folder_counts[dataset_folder_name],
        "Created_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
        "Start_Time": None,
        "End_Time": None,
        "GCS_Dataset_Path": gcs_dataset_path,
        "GCS_Experiment_Path": gcs_experiment_path,
    }

    insert_or_update_row(row_data, table_id)

    return f"Images uploaded successfully to {gcs_dir_path}."


def fetch_job_times(job_id: str) -> tuple:
    job = aiplatform.CustomJob.get(job_id)
    return job.create_time, job.start_time, job.end_time


def fetch_pipeline_job_times(job_id: str) -> tuple:
    pipeline_job = aiplatform.PipelineJob.get(job_id)
    while True:
        try:
            # Explore other attributes of the pipeline job
            pipeline_job_dict = pipeline_job.to_dict()
            create_time = pipeline_job_dict["createTime"].replace("Z", "")
            start_time = pipeline_job_dict["startTime"].replace("Z", "")
            update_time = pipeline_job_dict["updateTime"].replace("Z", "")
            break
        except KeyError:
            time.sleep(10)

    return create_time, start_time, update_time


def list_bq_folder_contents_colmap(table_id: str) -> dict:
    client = get_bigquery_client()
    query = f"""
        SELECT Experiment_ID, Scene_Name, Job_Status, Image_Count, Colmap_Job_ID, Created_Time, Start_Time, End_Time,
               Matcher_Type, Camera_Type, Video_Frame_FPS, Max_Num_Features, Use_Hierarchical_Mapper, GCS_Dataset_Path, GCS_Experiment_Path
        FROM `{table_id}`
        ORDER BY Experiment_ID
    """
    query_job = client.query(query)
    results = query_job.result()

    folder_counts = {}

    def process_row(row):
        if "Images Uploaded" in row.Job_Status:
            job_status = 0
        elif "nerf-pipeline" in row.Colmap_Job_ID:
            job_status = get_vertex_ai_pipeline_job_status(row.Colmap_Job_ID)
        elif "MANUAL" in row.Colmap_Job_ID:
            job_status = 4
        else:
            job_status = get_vertex_ai_job_status(row.Colmap_Job_ID)

        updated_job_status = JOB_STATE_MAPPING[int(job_status)]

        folder_counts[row.Scene_Name] = {
            "Experiment ID": row.Experiment_ID,
            "Job Status": updated_job_status,
            "Image Count": row.Image_Count,
            "Colmap Job ID": row.Colmap_Job_ID,
            "Created Time": (
                row.Created_Time.strftime("%Y-%m-%d %H:%M:%S")
                if row.Created_Time
                else None
            ),
            "Start Time": (
                row.Start_Time.strftime("%Y-%m-%d %H:%M:%S") if row.Start_Time else None
            ),
            "End Time": (
                row.End_Time.strftime("%Y-%m-%d %H:%M:%S") if row.End_Time else None
            ),
            "Matcher Type": row.Matcher_Type,
            "Camera Type": row.Camera_Type,
            "Video Frame FPS": row.Video_Frame_FPS,
            "Max Num Features": row.Max_Num_Features,
            "Use Hierarchical Mapper": row.Use_Hierarchical_Mapper,
            "GCS Dataset Path": row.GCS_Dataset_Path,
            "GCS Experiment Path": row.GCS_Experiment_Path,
        }

    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(process_row, results)

    return folder_counts


def list_bq_folder_contents_training(table_id: str) -> dict:
    client = get_bigquery_client()
    query = f"""
        SELECT Experiment_ID, Scene_Name, Job_Status, Image_Count, Colmap_Job_ID, Training_Job_ID, Training_Job_Name, Created_Time, Start_Time, End_Time,
               Training_Factor, Training_Max_Steps, ZipNeRF_Gin_Config, CamP_Gin_Config, GCS_Experiment_Path
        FROM `{table_id}`
        ORDER BY Experiment_ID
    """
    query_job = client.query(query)
    results = query_job.result()

    folder_counts = {}

    def process_row(row):
        if "NOT STARTED" in row.Job_Status:
            job_status = 0
        elif "nerf-pipeline" in row.Training_Job_ID:
            job_status = get_vertex_ai_pipeline_job_status(row.Training_Job_ID)
        else:
            job_status = get_vertex_ai_job_status(row.Training_Job_ID)
        updated_job_status = JOB_STATE_MAPPING[int(job_status)]
        folder_counts[row.Scene_Name] = {
            "Experiment ID": row.Experiment_ID,
            "Job Status": updated_job_status,
            "Image Count": row.Image_Count,
            "Colmap Job ID": row.Colmap_Job_ID,
            "Training Job ID": row.Training_Job_ID,
            "Training Job Name": row.Training_Job_Name,
            "Created Time": (
                row.Created_Time.strftime("%Y-%m-%d %H:%M:%S")
                if row.Created_Time
                else None
            ),
            "Start Time": (
                row.Start_Time.strftime("%Y-%m-%d %H:%M:%S") if row.Start_Time else None
            ),
            "End Time": (
                row.End_Time.strftime("%Y-%m-%d %H:%M:%S") if row.End_Time else None
            ),
            "Training Factor": row.Training_Factor,
            "Training Max Steps": row.Training_Max_Steps,
            "ZipNeRF Gin Config": row.ZipNeRF_Gin_Config,
            "CamP Gin Config": row.CamP_Gin_Config,
            "GCS Experiment Path": row.GCS_Experiment_Path,
        }

    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(process_row, results)

    return folder_counts


def list_bq_folder_contents_rendering(table_id: str) -> dict:
    client = get_bigquery_client()
    query = f"""
        SELECT Experiment_ID, Scene_Name, Job_Status, Image_Count, Colmap_Job_ID, Training_Job_ID, Training_Job_Name, Rendering_Job_ID, Rendering_Job_Name, Created_Time, Start_Time, End_Time,
               Render_Factor, Render_Resolution_Width, Render_Resolution_Height, Render_FPS, ZipNeRF_Gin_Config, CamP_Gin_Config,
               GCS_Keyframes_File, GCS_Render_Path_File, Render_Camtype, Render_Focal, Render_Path_Frames, GCS_Experiment_Path
        FROM `{table_id}`
        ORDER BY Experiment_ID
    """
    query_job = client.query(query)
    results = query_job.result()

    folder_counts = {}

    def process_row(row):
        if "NOT STARTED" in row.Job_Status:
            job_status = 0
        elif "nerf-pipeline" in row.Rendering_Job_ID:
            job_status = get_vertex_ai_pipeline_job_status(row.Rendering_Job_ID)
        else:
            job_status = get_vertex_ai_job_status(row.Rendering_Job_ID)
        updated_job_status = JOB_STATE_MAPPING[int(job_status)]
        folder_counts[row.Scene_Name] = {
            "Experiment ID": row.Experiment_ID,
            "Job Status": updated_job_status,
            "Image Count": row.Image_Count,
            "Colmap Job ID": row.Colmap_Job_ID,
            "Training Job ID": row.Training_Job_ID,
            "Training Job Name": row.Training_Job_Name,
            "Rendering Job ID": row.Rendering_Job_ID,
            "Rendering Job Name": row.Rendering_Job_Name,
            "Created Time": (
                row.Created_Time.strftime("%Y-%m-%d %H:%M:%S")
                if row.Created_Time
                else None
            ),
            "Start Time": (
                row.Start_Time.strftime("%Y-%m-%d %H:%M:%S") if row.Start_Time else None
            ),
            "End Time": (
                row.End_Time.strftime("%Y-%m-%d %H:%M:%S") if row.End_Time else None
            ),
            "Render Factor": row.Render_Factor,
            "Render Resolution Width": row.Render_Resolution_Width,
            "Render Resolution Height": row.Render_Resolution_Height,
            "Render FPS": row.Render_FPS,
            "ZipNeRF Gin Config": row.ZipNeRF_Gin_Config,
            "CamP Gin Config": row.CamP_Gin_Config,
            "GCS Keyframes File": row.GCS_Keyframes_File,
            "GCS Render Path File": row.GCS_Render_Path_File,
            "Render Camtype": row.Render_Camtype,
            "Render Focal": row.Render_Focal,
            "Render Path Frames": row.Render_Path_Frames,
            "GCS Experiment Path": row.GCS_Experiment_Path,
        }

    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(process_row, results)

    return folder_counts


def get_bq_folders_dataframe_colmap(table_id: str) -> pd.DataFrame:
    try:
        folder_counts = list_bq_folder_contents_colmap(table_id)
        data = [
            {
                "Experiment ID": info["Experiment ID"],
                "Scene Name": scene,
                "Job Status": info["Job Status"],
                "Image Count": info["Image Count"],
                "Colmap Job ID": info["Colmap Job ID"],
                "Matcher Type": info["Matcher Type"],
                "Camera Type": info["Camera Type"],
                "Video Frame FPS": info["Video Frame FPS"],
                "Max Num Features": info["Max Num Features"],
                "Use Hierarchical Mapper": info["Use Hierarchical Mapper"],
                "GCS Dataset Path": info["GCS Dataset Path"],
                "GCS Experiment Path": info["GCS Experiment Path"],
            }
            for scene, info in folder_counts.items()
        ]
        return pd.DataFrame(data).sort_values(by="Experiment ID").reset_index(drop=True)
    except Exception as e:
        print(e)
        return pd.DataFrame()


def get_bq_folders_dataframe_training(table_id: str) -> pd.DataFrame:
    try:
        folder_counts = list_bq_folder_contents_training(table_id)
        data = [
            {
                "Experiment ID": info["Experiment ID"],
                "Scene Name": scene,
                "Job Status": info["Job Status"],
                "Image Count": info["Image Count"],
                "Colmap Job ID": info["Colmap Job ID"],
                "Training Job ID": info["Training Job ID"],
                "Training Job Name": info["Training Job Name"],
                "Training Factor": info["Training Factor"],
                "Training Max Steps": info["Training Max Steps"],
                "ZipNeRF Gin Config": info["ZipNeRF Gin Config"],
                "CamP Gin Config": info["CamP Gin Config"],
                "GCS Experiment Path": info["GCS Experiment Path"],
            }
            for scene, info in folder_counts.items()
        ]
        return pd.DataFrame(data).sort_values(by="Experiment ID").reset_index(drop=True)
    except Exception as e:
        print(e)
        return pd.DataFrame()  # Return an empty DataFrame in case of an exception


def get_bq_folders_dataframe_rendering(table_id: str) -> pd.DataFrame:
    try:
        folder_counts = list_bq_folder_contents_rendering(table_id)
        data = [
            {
                "Experiment ID": info["Experiment ID"],
                "Scene Name": scene,
                "Job Status": info["Job Status"],
                "Image Count": info["Image Count"],
                "Colmap Job ID": info["Colmap Job ID"],
                "Training Job ID": info["Training Job ID"],
                "Training Job Name": info["Training Job Name"],
                "Rendering Job ID": info["Rendering Job ID"],
                "Rendering Job Name": info["Rendering Job Name"],
                "Render Factor": info["Render Factor"],
                "Render Resolution Width": info["Render Resolution Width"],
                "Render Resolution Height": info["Render Resolution Height"],
                "Render FPS": info["Render FPS"],
                "ZipNeRF Gin Config": info["ZipNeRF Gin Config"],
                "CamP Gin Config": info["CamP Gin Config"],
                "GCS Keyframes File": info["GCS Keyframes File"],
                "GCS Render Path File": info["GCS Render Path File"],
                "Render Camtype": info["Render Camtype"],
                "Render Focal": info["Render Focal"],
                "Render Path Frames": info["Render Path Frames"],
                "GCS Experiment Path": info["GCS Experiment Path"],
            }
            for scene, info in folder_counts.items()
        ]
        return pd.DataFrame(data).sort_values(by="Experiment ID").reset_index(drop=True)
    except Exception as e:
        print(e)
        return pd.DataFrame()  # Return an empty DataFrame in case of an exception


def upload_to_bq_colmap_table(
    experiment_id: str,
    scene_name: str,
    job_status: str,
    image_count: int,
    colmap_job_id: str,
    created_time: str,
    start_time: str,
    end_time: str,
    matcher_type: str,
    camera_type: str,
    video_frame_fps: int,
    max_num_features: int,
    use_hierarchical_mapper: bool,
    gcs_dataset_path: str,
    gcs_experiment_path: str,
    table_id: str,
):
    row_data = {
        "Experiment_ID": experiment_id,
        "Scene_Name": scene_name,
        "Job_Status": job_status,
        "Image_Count": int(image_count),
        "Colmap_Job_ID": colmap_job_id,
        "Created_Time": (
            datetime.strptime(created_time, "%Y-%m-%d %H:%M:%S")
            if created_time
            else None
        ),
        "Start_Time": (
            datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S") if start_time else None
        ),
        "End_Time": (
            datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S") if end_time else None
        ),
        "Matcher_Type": matcher_type,
        "Camera_Type": camera_type,
        "Video_Frame_FPS": int(video_frame_fps),
        "Max_Num_Features": int(max_num_features),
        "Use_Hierarchical_Mapper": use_hierarchical_mapper,
        "GCS_Dataset_Path": gcs_dataset_path,
        "GCS_Experiment_Path": gcs_experiment_path,
    }
    insert_or_update_row(row_data, table_id)


def upload_to_bq_training_table(
    experiment_id: str,
    scene_name: str,
    job_status: str,
    image_count: int,
    colmap_job_id: str,
    training_job_id: str,
    training_job_name: str,
    created_time: str,
    start_time: str,
    end_time: str,
    training_factor: int,
    training_max_steps: int,
    zipnerf_gin_config: str,
    camp_gin_config: str,
    gcs_experiment_path: str,
    table_id: str,
):
    row_data = {
        "Experiment_ID": experiment_id,
        "Scene_Name": scene_name,
        "Job_Status": job_status,
        "Image_Count": int(image_count),
        "Colmap_Job_ID": colmap_job_id,
        "Training_Job_ID": training_job_id,
        "Training_Job_Name": training_job_name,
        "Created_Time": (
            datetime.strptime(created_time, "%Y-%m-%d %H:%M:%S")
            if created_time
            else None
        ),
        "Start_Time": (
            datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S") if start_time else None
        ),
        "End_Time": (
            datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S") if end_time else None
        ),
        "Training_Factor": int(training_factor),
        "Training_Max_Steps": int(training_max_steps),
        "ZipNeRF_Gin_Config": zipnerf_gin_config,
        "CamP_Gin_Config": camp_gin_config,
        "GCS_Experiment_Path": gcs_experiment_path,
    }
    insert_or_update_row(row_data, table_id)


def upload_to_bq_rendering_table(
    experiment_id: str,
    scene_name: str,
    job_status: str,
    image_count: int,
    colmap_job_id: str,
    training_job_id: str,
    training_job_name: str,
    rendering_job_id: str,
    rendering_job_name: str,
    created_time: str,
    start_time: str,
    end_time: str,
    render_factor: int,
    render_resolution_width: int,
    render_resolution_height: int,
    render_fps: int,
    zipnerf_gin_config: str,
    camp_gin_config: str,
    gcs_keyframes_file: str,
    gcs_render_path_file: str,
    render_camtype: str,
    render_focal: float,
    render_path_frames: int,
    gcs_experiment_path: str,
    table_id: str,
):
    row_data = {
        "Experiment_ID": experiment_id,
        "Scene_Name": scene_name,
        "Job_Status": job_status,
        "Image_Count": int(image_count),
        "Colmap_Job_ID": colmap_job_id,
        "Training_Job_ID": training_job_id,
        "Training_Job_Name": training_job_name,
        "Rendering_Job_ID": rendering_job_id,
        "Rendering_Job_Name": rendering_job_name,
        "Created_Time": (
            datetime.strptime(created_time, "%Y-%m-%d %H:%M:%S")
            if created_time
            else None
        ),
        "Start_Time": (
            datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S") if start_time else None
        ),
        "End_Time": (
            datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S") if end_time else None
        ),
        "Render_Factor": int(render_factor),
        "Render_Resolution_Width": int(render_resolution_width),
        "Render_Resolution_Height": int(render_resolution_height),
        "Render_FPS": int(render_fps),
        "ZipNeRF_Gin_Config": zipnerf_gin_config,
        "CamP_Gin_Config": camp_gin_config,
        "GCS_Keyframes_File": gcs_keyframes_file,
        "GCS_Render_Path_File": gcs_render_path_file,
        "Render_Camtype": render_camtype,
        "Render_Focal": 1,
        "Render_Path_Frames": int(render_path_frames),
        "GCS_Experiment_Path": gcs_experiment_path,
    }
    insert_or_update_row(row_data, table_id)


def update_job_info(job_id, table_id, scene_name, experiment_id):
    while True:
        job_status = get_vertex_ai_job_status(job_id)
        create_time, start_time, end_time = fetch_job_times(job_id)
        row_data = {
            "Job_Status": JOB_STATE_MAPPING[job_status],
            "Experiment_ID": experiment_id,
            "Scene_Name": scene_name,
            "Created_Time": create_time,
            "Start_Time": start_time,
            "End_Time": end_time,
        }
        insert_or_update_row(row_data, table_id)
        if int(job_status) in [0, 4, 5, 7, 8, 9]:
            break
        time.sleep(60)


def update_pipeline_job_info(job_id, table_id, scene_name, experiment_id, mode=1):
    while True:
        job_status = get_vertex_ai_pipeline_job_status(job_id)
        create_time, start_time, end_time = fetch_pipeline_job_times(job_id)
        row_data = {
            "Job_Status": JOB_STATE_MAPPING[job_status],
            "Experiment_ID": experiment_id,
            "Scene_Name": scene_name,
            "Created_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
            "Start_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
        }
        if mode == 1:
            row_data["Colmap_Job_ID"] = job_id
        elif mode == 2:
            row_data["Colmap_Job_ID"] = job_id
            row_data["Training_Job_ID"] = job_id
        else:
            row_data["Colmap_Job_ID"] = job_id
            row_data["Training_Job_ID"] = job_id
            row_data["Rendering_Job_ID"] = job_id
        insert_or_update_row(row_data, table_id)
        if int(job_status) in [0, 4, 5, 7, 8, 9]:
            row_data = {
                "Job_Status": JOB_STATE_MAPPING[job_status],
                "Experiment_ID": experiment_id,
                "Scene_Name": scene_name,
                "End_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
            }
            insert_or_update_row(row_data, table_id)
            break
        time.sleep(60)


# Function to delete selected row in BigQuery and GCS
def delete_experiment_row(selected_row, table_id):
    client = get_bigquery_client()

    # Delete row from BigQuery
    query = f"""
        DELETE FROM `{table_id}`
        WHERE Experiment_ID = '{selected_row}'
    """
    query_job = client.query(query)
    query_job.result()

    return f"Experiment {selected_row} deleted successfully."


def create_pipeline(
    calibration_job_name: str,
    training_job_name: str,
    rendering_job_name: str,
    calibration_worker_pool_specs,
    training_worker_pool_specs,
    rendering_worker_pool_specs,
    notification_emails=None,
):
    @dsl.pipeline
    def nerf_pipeline():
        notify_email_task = VertexNotificationEmailOp(
            recipients=notification_emails
        ).set_display_name("Notify email")
        with dsl.ExitHandler(notify_email_task, name="CamP ZipNeRF Pipeline"):
            camera_pose_task = CustomTrainingJobOp(
                display_name=calibration_job_name,
                worker_pool_specs=calibration_worker_pool_specs,
            )
            camera_pose_task.set_display_name(calibration_job_name)

            train_zipnerf_task = CustomTrainingJobOp(
                display_name=training_job_name,
                worker_pool_specs=training_worker_pool_specs,
            ).after(camera_pose_task)
            train_zipnerf_task.set_display_name(training_job_name)

            render_zipnerf_task = CustomTrainingJobOp(
                display_name=rendering_job_name,
                worker_pool_specs=rendering_worker_pool_specs,
            ).after(train_zipnerf_task)
            render_zipnerf_task.set_display_name(rendering_job_name)

    pipeline_name = "lightweight_pipeline.yaml"
    compiler.Compiler().compile(pipeline_func=nerf_pipeline, package_path=pipeline_name)
    return pipeline_name


def run_pipeline(pipeline_name: str):
    PIPELINE_ROOT = (
        f"{BUCKET_NAME}/pipeline_root/{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    )
    DISPLAY_NAME = get_job_name_with_datetime(
        "NeRFGradio_" + generate_short_unique_identifier()
    )
    job = aiplatform.PipelineJob(
        display_name=DISPLAY_NAME,
        template_path=pipeline_name,
        pipeline_root=PIPELINE_ROOT,
    )
    job.run(sync=False)
    return job

In [39]:
# @title Pose Estimation Workshop


def create_pose_estimation_workshop():

    def get_worker_pool_specs(docker_uri, args, machine_type, accelerator_type):
        return [
            {
                "machine_spec": {
                    "machine_type": machine_type,
                    "accelerator_type": accelerator_type,
                    "accelerator_count": 8,
                },
                "replica_count": 1,
                "container_spec": {
                    "image_uri": docker_uri,
                    "args": args,
                },
            }
        ]

    def extract_frames_from_video(video_path: str, dest_dir: str, frame_rate: int = 1):
        clip = VideoFileClip(video_path)
        duration = clip.duration
        total_frames = int(duration * frame_rate)

        for t in range(total_frames):
            frame = clip.get_frame(t / frame_rate)
            frame_image = Image.fromarray(frame)
            frame_image.save(
                os.path.join(dest_dir, f"frame_{t + 1}.jpg"), format="JPEG", quality=100
            )

    def prepare_instance_images(
        scene_name: str,
        experiment_name: str,
        file_collection: List[gr.File],
        frame_rate=1,
        progress=gr.Progress(),
    ):
        if not file_collection:
            raise gr.Error("Please provide a few valid instance images first!")

        local_tmp_dir = "/tmp/instance_images"
        if os.path.exists(local_tmp_dir):
            shutil.rmtree(local_tmp_dir)
        os.makedirs(local_tmp_dir, exist_ok=True)

        total_files = len(file_collection)
        completed_files = 0

        for i, file_temp in enumerate(file_collection, start=1):
            file_ext = os.path.splitext(file_temp.name)[1].lower()

            if file_ext in [
                ".jpg",
                ".jpeg",
                ".png",
                ".bmp",
                ".gif",
            ]:  # Image file extensions
                image = Image.open(file_temp.name).convert("RGB")
                image.save(
                    os.path.join(local_tmp_dir, f"image_{i}.jpg"),
                    format="JPEG",
                    quality=100,
                )
            elif file_ext in [".mp4", ".avi", ".mov", ".mkv"]:  # Video file extensions
                extract_frames_from_video(file_temp.name, local_tmp_dir, frame_rate)

            completed_files += 1
            progress(completed_files / total_files)

        instant_image_dir = os.path.join(GCS_API_ENDPOINT, experiment_name)
        upload_local_dir_to_gcs(
            scene_name, local_tmp_dir, instant_image_dir, table_id=colmap_table_id
        )

    def prepare_instance_images_from_gcs(
        scene_name: str, experiment_name: str, gcs_folder: str, progress=gr.Progress()
    ):

        gcs_folder_path = gcs_folder.replace("gs://", "")
        path_bucket_name = gcs_folder_path.split("/")[0]
        folder_path = gcs_folder_path.replace(path_bucket_name + "/", "")
        output_experiment_name = experiment_name.replace("dataset", "experiment")
        gcs_experiment_path = os.path.join(BUCKET_NAME, output_experiment_name)

        client = storage.Client()
        bucket = client.get_bucket(path_bucket_name)
        blobs = bucket.list_blobs(prefix=folder_path)

        # Initialize a set to track unique image names
        unique_images = set()
        unique_videos = set()

        # Loop through the blobs and add unique image names to the set
        for blob in blobs:
            # Get the MIME type of the file
            mime_type, _ = mimetypes.guess_type(blob.name)
            image_name = blob.name.split("/")[-1]
            if mime_type and mime_type.startswith("image"):
                # Extract the image name (without the folder path)
                unique_images.add(image_name)
            else:
                unique_videos.add(image_name)

        # Return the count of unique images
        file_count = len(unique_images) + len(unique_videos)

        scene_name = scene_name + "_" + generate_short_unique_identifier()

        row_data = {
            "Experiment_ID": experiment_name,
            "Scene_Name": scene_name,
            "Job_Status": "Images Uploaded",
            "Colmap_Job_ID": "",
            "Image_Count": file_count,
            "Created_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
            "Start_Time": None,
            "End_Time": None,
            "GCS_Dataset_Path": gcs_folder,
            "GCS_Experiment_Path": gcs_experiment_path,
        }
        insert_or_update_row(row_data, colmap_table_id)
        return f"{gcs_folder} dataset successfully inserted to database."

    def upload_images_to_gcs(
        scene_name,
        experiment_name,
        file_collection,
        gcs_folder,
        frame_rate=1,
        progress=gr.Progress(),
    ):
        if gcs_folder:
            prepare_instance_images_from_gcs(
                scene_name, experiment_name, gcs_folder, progress
            )
        else:
            prepare_instance_images(
                scene_name, experiment_name, file_collection, frame_rate, progress
            )
        return get_bq_folders_dataframe_colmap(colmap_table_id)

    def start_colmap(
        selected_row,
        matcher_dropdown,
        camera_dropdown,
        video_frame_fps,
        max_num_features,
        mapper_dropdown,
        machine_dropdown,
    ):
        print("Launching colmap...")
        selected_row = extract_dataset_id(selected_row)
        folders_df = get_bq_folders_dataframe_colmap(colmap_table_id)
        gcs_dataset_path = folders_df[folders_df["Experiment ID"] == selected_row][
            "GCS Dataset Path"
        ].iloc[0]

        input_image_folder = f"{BUCKET_NAME}/{selected_row}"
        output_dir = input_image_folder.replace("dataset", "experiment")
        data_calibration_job_name = get_job_name_with_datetime(
            "cloudnerf_gradio_colmap"
        )
        unique_experiments.add(selected_row)

        machine_type = (
            "n1-highmem-64"
            if machine_dropdown == "NVIDIA_TESLA_V100"
            else "a2-highgpu-8g"
        )
        accelerator_type = machine_dropdown
        accelerator_count = 8

        colmap_args = [
            "-use_gpu",
            "1",
            "-gcs_dataset_path",
            gcs_dataset_path,
            "-gcs_experiment_path",
            output_dir,
            "-camera",
            camera_dropdown,
            "-matching_strategy",
            MATCHER_MAPPING[matcher_dropdown],
            "-max_num_features",
            str(max_num_features),
            "-use_hierarchical_mapper",
            str(int(mapper_dropdown)),
            "-fps",
            str(video_frame_fps),
        ]

        worker_pool_specs = get_worker_pool_specs(
            CALIBRATION_DOCKER_URI, colmap_args, machine_type, accelerator_type
        )

        data_calibration_custom_job = aiplatform.CustomJob(
            display_name=data_calibration_job_name,
            project=PROJECT_ID,
            worker_pool_specs=worker_pool_specs,
            staging_bucket=staging_bucket,
        )

        data_calibration_custom_job.run(sync=False)

        scene_name = folders_df[folders_df["Experiment ID"] == selected_row][
            "Scene Name"
        ].iloc[0]
        image_count = int(
            folders_df[folders_df["Experiment ID"] == selected_row]["Image Count"].iloc[
                0
            ]
        )

        colmap_row_data = {
            "Job_Status": "RUNNING",
            "Experiment_ID": selected_row,
            "Scene_Name": scene_name,
            "Image_Count": image_count,
            "Colmap_Job_ID": "STARTING",
            "Created_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
            "Matcher_Type": MATCHER_MAPPING[matcher_dropdown],
            "Camera_Type": camera_dropdown,
            "Video_Frame_FPS": video_frame_fps,
            "Max_Num_Features": max_num_features,
            "Use_Hierarchical_Mapper": mapper_dropdown,
            "GCS_Dataset_Path": gcs_dataset_path,
            "GCS_Experiment_Path": output_dir,
            "Machine_Type": machine_type,
            "Accelerator_Type": accelerator_type,
            "Accelerator_Count": accelerator_count,
        }

        training_row_data = {
            "Job_Status": "NOT STARTED",
            "Experiment_ID": selected_row,
            "Scene_Name": scene_name,
            "Image_Count": image_count,
            "Colmap_Job_ID": "STARTING",
            "Training_Job_ID": "NOT STARTED",
            "Training_Job_Name": "",
            "Created_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
            "GCS_Experiment_Path": output_dir,
            "Machine_Type": machine_type,
            "Accelerator_Type": accelerator_type,
            "Accelerator_Count": accelerator_count,
        }

        insert_or_update_row(colmap_row_data, colmap_table_id)
        insert_or_update_row(training_row_data, training_table_id)

        while True:
            try:
                if data_calibration_custom_job.resource_name:
                    colmap_job_id = data_calibration_custom_job.resource_name.split(
                        "/"
                    )[-1]
                    colmap_job_status = get_vertex_ai_job_status(colmap_job_id)
                    colmap_status_thread = threading.Thread(
                        target=update_job_info,
                        args=(
                            str(colmap_job_id),
                            colmap_table_id,
                            scene_name,
                            selected_row,
                        ),
                    )
                    colmap_status_thread.start()
                    colmap_row_data.update(
                        {
                            "Job_Status": JOB_STATE_MAPPING[int(colmap_job_status)],
                            "Colmap_Job_ID": colmap_job_id,
                            "Start_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
                        }
                    )
                    training_row_data.update(
                        {
                            "Colmap_Job_ID": colmap_job_id,
                        }
                    )
                    insert_or_update_row(colmap_row_data, colmap_table_id)
                    insert_or_update_row(training_row_data, training_table_id)
                    break
            except RuntimeError:
                pass
        return get_bq_folders_dataframe_colmap(colmap_table_id)

    camp_zipnerf_tip_text = """
        1. Upload images or videos.
        2. Set parameters for COLMAP.
        3. Select dataset in the table.
        4. Click the **Run Colmap** button.
        5. After the colmap job starts, check the job status at
           [Vertex Custom Training](https://console.cloud.google.com/vertex-ai/training/custom-jobs?project={PROJECT_ID}).
        """

    def create_dropdown(label, options, default):
        return gr.Dropdown(options, label=label, interactive=True, value=default)

    def create_textbox(label, default, visible=True):
        return gr.Textbox(
            label=label, value=default, lines=1, interactive=True, visible=visible
        )

    def create_number(label, default):
        return gr.Number(label=label, value=default, interactive=True)

    with gr.Blocks() as demo:

        def update_experiment_and_dataset_names(dataset_name, experiment_path):
            datetime_suffix = datetime.now().strftime("_%Y%m%d_%H%M%S")
            if datetime_suffix not in dataset_name:
                dataset_name = "dataset" + datetime_suffix
            if datetime_suffix not in experiment_path:
                experiment_path = os.path.join(
                    BUCKET_NAME, "experiment" + datetime_suffix
                )
            return dataset_name, experiment_path

        def update_names(scene_name):
            dataset_name, experiment_path = update_experiment_and_dataset_names(
                scene_name, ""
            )
            return dataset_name, experiment_path

        def delete_selected_experiment(selected_value):
            selected_value = extract_dataset_id(selected_value)
            delete_experiment_row(selected_value, colmap_table_id)
            delete_experiment_row(selected_value, training_table_id)
            delete_experiment_row(selected_value, rendering_table_id)
            return get_bq_folders_dataframe_colmap(colmap_table_id)

        def on_row_select(folders_df, evt: gr.SelectData):
            row_index = evt.index[0]
            if 0 <= row_index < len(folders_df):
                selected_value = folders_df.iloc[row_index, 0]
                colmap_job_id = folders_df[
                    folders_df["Experiment ID"] == selected_value
                ]["Colmap Job ID"].iloc[0]
                job_status = folders_df[folders_df["Experiment ID"] == selected_value][
                    "Job Status"
                ].iloc[0]
                colmap_job_link = selected_value
                if job_status not in ["NOT STARTED"]:
                    if "nerf-pipeline" in colmap_job_id:
                        colmap_job_link = get_vertex_ai_pipeline_run_link(
                            colmap_job_id, PROJECT_NUMBER, REGION
                        )
                        colmap_job_link = f"[{selected_value}]({colmap_job_link})"
                    elif colmap_job_id == "MANUAL":
                        pass
                    else:
                        colmap_job_link = get_vertex_ai_training_job_link(
                            colmap_job_id, PROJECT_NUMBER, REGION
                        )
                        colmap_job_link = f"[{selected_value}]({colmap_job_link})"
                return colmap_job_link, gr.update(visible=True), gr.update(visible=True)
            else:
                return "", gr.update(visible=False), gr.update(visible=True)

        with gr.Accordion("How To Use", open=False):
            gr.Markdown(camp_zipnerf_tip_text)

        with gr.Accordion("Datasets", open=True):
            folders_dataframe = gr.Dataframe(
                value=get_bq_folders_dataframe_colmap(colmap_table_id),
                interactive=False,
            )
            selected_folder = gr.Markdown()
            with gr.Row(equal_height=True):
                run_colmap_button = gr.Button("RUN COLMAP", visible=False)
                delete_button = gr.Button("DELETE", visible=False)
            folders_dataframe.select(
                on_row_select,
                inputs=[folders_dataframe],
                outputs=[selected_folder, run_colmap_button, delete_button],
            )

        with gr.Row(equal_height=True):
            with gr.Column():
                gr.Markdown("### UPLOAD NEW DATASET")
                scene_name = create_textbox("Scene Name", "")
                experiment_name = create_textbox(
                    "Experiment Name",
                    get_job_name_with_datetime("dataset"),
                    visible=False,
                )
                output_dir = create_textbox(
                    "Output Directory",
                    get_job_name_with_datetime("experiment"),
                    visible=False,
                )
                file_collection = gr.File(
                    label="Upload the images or video for your NeRF.",
                    file_types=["image", "video"],
                    file_count="multiple",
                    interactive=True,
                    visible=True,
                )
                gcs_folder = create_textbox("GCS Folder with images or video", "")
                upload_images_button = gr.Button(
                    "Upload Scene to GCS", variant="primary"
                )
                _ = gr.Markdown(visible=False)

            with gr.Column():
                gr.Markdown("### SET COLMAP SETTINGS")
                matcher_dropdown = create_dropdown(
                    "Choose a Matching Algorithm",
                    [
                        "Exhaustive Matcher",
                        "Sequential Matcher",
                        "Spatial Matcher",
                        "Transitive Matcher",
                        "Vocab Tree Matcher",
                    ],
                    "Exhaustive Matcher",
                )
                camera_dropdown = create_dropdown(
                    "Type of Camera Used for Capture",
                    ["OPENCV", "OPENCV_FISHEYE"],
                    "OPENCV",
                )
                machine_dropdown = create_dropdown(
                    "Select Machine Type",
                    ["NVIDIA_TESLA_V100", "NVIDIA_TESLA_A100"],
                    "NVIDIA_TESLA_V100",
                )
                with gr.Accordion("Advanced Settings", open=False):
                    video_frame_fps = create_number("Video Frame Extraction FPS", 4)
                    max_num_features = create_number(
                        "Maximum Number Length of SIFT Descriptor", 8192
                    )
                    mapper_dropdown = create_dropdown(
                        "Use Hierarchical Mapper", [True, False], False
                    )

        ws_table_id = create_textbox(
            "Workspace Table ID", colmap_table_id, visible=False
        )

        upload_images_button.click(
            upload_images_to_gcs,
            inputs=[
                scene_name,
                experiment_name,
                file_collection,
                gcs_folder,
                video_frame_fps,
            ],
            outputs=[folders_dataframe],
            show_progress=True,
            concurrency_limit=10,
        )

        run_colmap_button.click(
            start_colmap,
            inputs=[
                selected_folder,
                matcher_dropdown,
                camera_dropdown,
                video_frame_fps,
                max_num_features,
                mapper_dropdown,
                machine_dropdown,
            ],
            outputs=[folders_dataframe],
            show_progress=True,
            concurrency_limit=10,
        )

        delete_button.click(
            delete_selected_experiment,
            inputs=[selected_folder],
            outputs=[folders_dataframe],
            concurrency_limit=10,
        )

        demo.load(
            get_bq_folders_dataframe_colmap,
            inputs=[ws_table_id],
            outputs=[folders_dataframe],
            concurrency_limit=10,
        )

        scene_name.change(
            update_names, inputs=[scene_name], outputs=[experiment_name, output_dir]
        )

        experiment_name.change(
            update_experiment_and_dataset_names,
            inputs=[experiment_name, output_dir],
            outputs=[experiment_name, output_dir],
        )

        output_dir.change(
            update_experiment_and_dataset_names,
            inputs=[experiment_name, output_dir],
            outputs=[experiment_name, output_dir],
        )

        demo.load(
            update_experiment_and_dataset_names,
            inputs=[experiment_name, output_dir],
            outputs=[experiment_name, output_dir],
        )

        return demo, folders_dataframe

In [40]:
# @title Training Workshop


def create_training_workshop():

    def get_worker_pool_specs(docker_uri, args, machine_type, accelerator_type):
        return [
            {
                "machine_spec": {
                    "machine_type": machine_type,
                    "accelerator_type": accelerator_type,
                    "accelerator_count": 8,
                },
                "replica_count": 1,
                "container_spec": {
                    "image_uri": docker_uri,
                    "args": args,
                },
            }
        ]

    def update_colmap_dataset(selected_row, colmap_gcs_folder):
        print("Updating pre-processed colmap dataset...")
        training_df = get_bq_folders_dataframe_training(training_table_id)
        selected_row = extract_dataset_id(selected_row)
        filtered_df = training_df[training_df["Experiment ID"] == selected_row]
        if not filtered_df.empty:
            scene_name = filtered_df["Scene Name"].iloc[0]
            colmap_row_data = {
                "Job_Status": "SUCCEEDED",
                "Scene_Name": scene_name,
                "Experiment_ID": selected_row,
                "Colmap_Job_ID": "MANUAL",
                "GCS_Experiment_Path": colmap_gcs_folder,
                "End_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
            }

            training_row_data = {
                "Experiment_ID": selected_row,
                "Scene_Name": scene_name,
                "Colmap_Job_ID": "MANUAL",
                "GCS_Experiment_Path": colmap_gcs_folder,
            }

            insert_or_update_row(colmap_row_data, colmap_table_id)
            insert_or_update_row(training_row_data, training_table_id)
        _ = get_bq_folders_dataframe_colmap(colmap_table_id)
        return get_bq_folders_dataframe_training(training_table_id)

    def start_training(
        selected_folder,
        training_factor,
        training_max_steps,
        zipnerf_gin_config,
        camp_gin_config,
        machine_dropdown,
    ):
        print("Launching training...")
        selected_folder = extract_dataset_id(selected_folder)
        colmap_df = get_bq_folders_dataframe_colmap(colmap_table_id)
        output_colmap_dir_gcs = colmap_df[
            colmap_df["Experiment ID"] == selected_folder
        ]["GCS Experiment Path"].iloc[0]
        data_training_job_name = get_job_name_with_datetime("cloudnerf_gradio_training")
        unique_experiments.add(selected_folder)

        machine_type = (
            "n1-highmem-64"
            if machine_dropdown == "NVIDIA_TESLA_V100"
            else "a2-highgpu-8g"
        )
        accelerator_type = machine_dropdown
        accelerator_count = 8

        training_args = [
            "-training_job_name",
            data_training_job_name,
            "-gcs_experiment_path",
            output_colmap_dir_gcs,
            "-factor",
            str(training_factor),
            "-max_steps",
            str(training_max_steps),
            "-gin_config_zipnerf",
            zipnerf_gin_config,
            "-gin_config_camp",
            camp_gin_config,
        ]

        worker_pool_specs = get_worker_pool_specs(
            TRAINING_DOCKER_URI, training_args, machine_type, accelerator_type
        )

        data_training_custom_job = aiplatform.CustomJob(
            display_name=data_training_job_name,
            project=PROJECT_ID,
            worker_pool_specs=worker_pool_specs,
            staging_bucket=staging_bucket,
        )

        data_training_custom_job.run(sync=False)

        filtered_df = colmap_df[colmap_df["Experiment ID"] == selected_folder]

        if not filtered_df.empty:
            colmap_job_id = filtered_df["Colmap Job ID"].iloc[0]
        else:
            colmap_job_id = "PENDING"

        if not filtered_df.empty:
            colmap_job_status = filtered_df["Job Status"].iloc[0]
        else:
            colmap_job_status = "PENDING"

        if not filtered_df.empty:
            scene_name = filtered_df["Scene Name"].iloc[0]
        else:
            scene_name = "NOT FOUND"

        if not filtered_df.empty:
            image_count = int(filtered_df["Image Count"].iloc[0])
        else:
            image_count = 0

        colmap_row_data = {
            "Job_Status": colmap_job_status,
            "Experiment_ID": selected_folder,
            "Scene_Name": scene_name,
            "Image_Count": image_count,
            "Colmap_Job_ID": colmap_job_id,
            "End_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
        }

        training_row_data = {
            "Job_Status": "RUNNING",
            "Experiment_ID": selected_folder,
            "Scene_Name": scene_name,
            "Image_Count": image_count,
            "Colmap_Job_ID": colmap_job_id,
            "Training_Job_ID": "STARTING",
            "Training_Job_Name": data_training_job_name,
            "Start_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
            "Training_Factor": training_factor,
            "Training_Max_Steps": training_max_steps,
            "ZipNeRF_Gin_Config": zipnerf_gin_config,
            "CamP_Gin_Config": camp_gin_config,
            "GCS_Experiment_Path": output_colmap_dir_gcs,
            "Machine_Type": machine_type,
            "Accelerator_Type": accelerator_type,
            "Accelerator_Count": accelerator_count,
        }

        rendering_row_data = {
            "Job_Status": "NOT STARTED",
            "Experiment_ID": selected_folder,
            "Scene_Name": scene_name,
            "Image_Count": image_count,
            "Colmap_Job_ID": colmap_job_id,
            "Training_Job_ID": "STARTING",
            "Training_Job_Name": data_training_job_name,
            "Rendering_Job_ID": "NOT STARTED",
            "Created_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
            "GCS_Experiment_Path": output_colmap_dir_gcs,
            "Machine_Type": machine_type,
            "Accelerator_Type": accelerator_type,
            "Accelerator_Count": accelerator_count,
        }

        insert_or_update_row(colmap_row_data, colmap_table_id)
        insert_or_update_row(training_row_data, training_table_id)
        insert_or_update_row(rendering_row_data, rendering_table_id)

        while True:
            try:
                if data_training_custom_job.resource_name:
                    training_job_id = data_training_custom_job.resource_name.split("/")[
                        -1
                    ]
                    training_job_status = get_vertex_ai_job_status(training_job_id)
                    training_status_thread = threading.Thread(
                        target=update_job_info,
                        args=(
                            str(training_job_id),
                            training_table_id,
                            scene_name,
                            selected_folder,
                        ),
                    )
                    training_status_thread.start()
                    training_row_data.update(
                        {
                            "Job_Status": JOB_STATE_MAPPING[int(training_job_status)],
                            "Training_Job_ID": training_job_id,
                        }
                    )
                    rendering_row_data.update(
                        {
                            "Training_Job_ID": training_job_id,
                        }
                    )
                    insert_or_update_row(training_row_data, training_table_id)
                    insert_or_update_row(rendering_row_data, rendering_table_id)
                    break
            except RuntimeError:
                pass

        return get_bq_folders_dataframe_training(training_table_id)

    training_tip_text = """
        1. Upload images or videos in the Pose Estimation tab.
        2. Set parameters for training.
        3. Select dataset in the table.
        4. Click the **Run Training** button.
        5. After the rendering job starts, check the job status at
           [Vertex Custom Training](https://console.cloud.google.com/vertex-ai/training/custom-jobs?project={PROJECT_ID}).
        """

    def create_dropdown(label, options, default):
        return gr.Dropdown(options, label=label, interactive=True, value=default)

    def create_textbox(label, default, visible=True):
        return gr.Textbox(
            label=label, value=default, lines=1, interactive=True, visible=visible
        )

    def create_number(label, default):
        return gr.Number(label=label, value=default, interactive=True)

    with gr.Blocks() as demo:

        with gr.Accordion("How To Use", open=False):
            gr.Markdown(training_tip_text)

        def on_row_select(training_df, evt: gr.SelectData):
            row_index = evt.index[0]
            if 0 <= row_index < len(training_df):
                selected_value = training_df.iloc[row_index, 0]
                training_job_id = training_df[
                    training_df["Experiment ID"] == selected_value
                ]["Training Job ID"].iloc[0]
                job_status = training_df[
                    training_df["Experiment ID"] == selected_value
                ]["Job Status"].iloc[0]
                training_job_link = selected_value
                if job_status not in ["NOT STARTED"]:
                    if "nerf-pipeline" in training_job_id:
                        training_job_link = get_vertex_ai_pipeline_run_link(
                            training_job_id, PROJECT_NUMBER, REGION
                        )
                        training_job_link = f"[{selected_value}]({training_job_link})"
                    elif training_job_id == "MANUAL":
                        pass
                    else:
                        training_job_link = get_vertex_ai_training_job_link(
                            training_job_id, PROJECT_NUMBER, REGION
                        )
                        training_job_link = f"[{selected_value}]({training_job_link})"
                return training_job_link, gr.update(visible=True)
            else:
                return "", gr.update(visible=False)

        with gr.Accordion("Colmap Datasets", open=True):
            training_dataframe = gr.Dataframe(
                value=get_bq_folders_dataframe_training(training_table_id),
                interactive=False,
            )
            selected_folder = gr.Markdown()
            run_training_button = gr.Button("RUN TRAINING", visible=False)
            training_dataframe.select(
                on_row_select,
                inputs=[training_dataframe],
                outputs=[selected_folder, run_training_button],
            )

        with gr.Row(equal_height=True):
            with gr.Column():
                gr.Markdown("### SET PROCESSED COLMAP DATASET")
                colmap_gcs_folder = create_textbox("Enter GCS Experiment Folder", "")
                set_processed_colmap_button = gr.Button(
                    "Set Processed COLMAP Data", variant="primary"
                )
            with gr.Column():
                gr.Markdown("### SET TRAINING PARAMETERS")
                training_factor = create_dropdown("Downscaling Factor", [0, 2, 4, 8], 4)
                training_max_steps = create_number(
                    "Maximum Number of Training Steps", 25000
                )
                machine_dropdown = create_dropdown(
                    "Select Machine Type",
                    ["NVIDIA_TESLA_V100", "NVIDIA_TESLA_A100"],
                    "NVIDIA_TESLA_V100",
                )
                with gr.Accordion("Advanced Settings", open=False):
                    zipnerf_gin_config = create_textbox(
                        "The ZipNeRF .gin Configuration File",
                        "configs/zipnerf/360_aglo128.gin",
                    )
                    camp_gin_config = create_textbox(
                        "The CamP .gin Configuration File",
                        "configs/camp/camera_optim.gin",
                    )

            ws_table_id = create_textbox(
                "Workspace Table ID", training_table_id, visible=False
            )

        run_training_button.click(
            start_training,
            inputs=[
                selected_folder,
                training_factor,
                training_max_steps,
                zipnerf_gin_config,
                camp_gin_config,
                machine_dropdown,
            ],
            outputs=[training_dataframe],
            show_progress=True,
            concurrency_limit=10,
        )

        set_processed_colmap_button.click(
            update_colmap_dataset,
            inputs=[selected_folder, colmap_gcs_folder],
            outputs=[training_dataframe],
            show_progress=True,
            concurrency_limit=10,
        )

        demo.load(
            get_bq_folders_dataframe_training,
            inputs=[ws_table_id],
            outputs=[training_dataframe],
            concurrency_limit=10,
        )

        return demo, training_dataframe

In [41]:
# @title Rendering Workshop


def create_rendering_workshop():

    def get_worker_pool_specs(docker_uri, args, machine_type, accelerator_type):
        return [
            {
                "machine_spec": {
                    "machine_type": machine_type,
                    "accelerator_type": accelerator_type,
                    "accelerator_count": 8,
                },
                "replica_count": 1,
                "container_spec": {
                    "image_uri": docker_uri,
                    "args": args,
                },
            }
        ]

    def start_rendering(
        selected_folder,
        gcs_keyframes_file,
        render_factor,
        render_resolution_width,
        render_resolution_height,
        render_fps,
        zipnerf_gin_config,
        camp_gin_config,
        gcs_render_path_file,
        render_camtype,
        render_path_frames,
        machine_dropdown,
    ):
        print("Launching rendering...")
        rendering_df = get_bq_folders_dataframe_rendering(rendering_table_id)
        training_df = get_bq_folders_dataframe_training(training_table_id)

        selected_folder = extract_dataset_id(selected_folder)
        output_colmap_dir_gcs = rendering_df[
            rendering_df["Experiment ID"] == selected_folder
        ]["GCS Experiment Path"].iloc[0]
        training_job_name = rendering_df[
            rendering_df["Experiment ID"] == selected_folder
        ]["Training Job Name"].iloc[0]

        data_rendering_job_name = get_job_name_with_datetime(
            "cloudnerf_gradio_rendering"
        )
        unique_experiments.add(selected_folder)
        video_resolution = f"({render_resolution_width}, {render_resolution_height})"

        machine_type = (
            "n1-highmem-64"
            if machine_dropdown == "NVIDIA_TESLA_V100"
            else "a2-highgpu-8g"
        )
        accelerator_type = machine_dropdown
        accelerator_count = 8

        rendering_args = [
            "-rendering_job_name",
            data_rendering_job_name,
            "-training_job_name",
            training_job_name,
            "-gcs_experiment_path",
            output_colmap_dir_gcs,
            "-render_resolution",
            video_resolution,
            "-render_video_fps",
            str(render_fps),
            "-factor",
            str(render_factor),
            "-gin_config_zipnerf",
            zipnerf_gin_config,
            "-gin_config_camp",
            camp_gin_config,
        ]
        if gcs_keyframes_file:
            rendering_args.append("-gcs_keyframes_file")
            rendering_args.append(gcs_keyframes_file)
        if render_camtype:
            rendering_args.append("-render_camtype")
            rendering_args.append(render_camtype)
        if gcs_render_path_file and not gcs_keyframes_file:
            rendering_args.append("-gcs_render_path_file")
            rendering_args.append(gcs_render_path_file)

        worker_pool_specs = get_worker_pool_specs(
            RENDERING_DOCKER_URI, rendering_args, machine_type, accelerator_type
        )

        data_rendering_custom_job = aiplatform.CustomJob(
            display_name=data_rendering_job_name,
            project=PROJECT_ID,
            worker_pool_specs=worker_pool_specs,
            staging_bucket=staging_bucket,
        )

        data_rendering_custom_job.run(sync=False)

        filtered_df = training_df[training_df["Experiment ID"] == selected_folder]

        if not filtered_df.empty:
            colmap_job_id = filtered_df["Colmap Job ID"].iloc[0]
        else:
            colmap_job_id = "PENDING"

        if not filtered_df.empty:
            training_job_id = filtered_df["Training Job ID"].iloc[0]
        else:
            training_job_id = "PENDING"

        if not filtered_df.empty:
            training_job_status = filtered_df["Job Status"].iloc[0]
        else:
            training_job_status = "PENDING"

        if not filtered_df.empty:
            scene_name = filtered_df["Scene Name"].iloc[0]
        else:
            scene_name = "NOT FOUND"

        if not filtered_df.empty:
            image_count = int(filtered_df["Image Count"].iloc[0])
        else:
            image_count = 0

        training_row_data = {
            "Job_Status": training_job_status,
            "Experiment_ID": selected_folder,
            "Scene_Name": scene_name,
            "Image_Count": image_count,
            "Colmap_Job_ID": colmap_job_id,
            "Training_Job_ID": training_job_id,
            "Training_Job_Name": training_job_name,
            "End_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
            "GCS_Experiment_Path": output_colmap_dir_gcs,
        }

        rendering_row_data = {
            "Job_Status": "RUNNING",
            "Experiment_ID": selected_folder,
            "Scene_Name": scene_name,
            "Image_Count": image_count,
            "Colmap_Job_ID": colmap_job_id,
            "Training_Job_ID": training_job_id,
            "Training_Job_Name": training_job_name,
            "Rendering_Job_ID": "STARTING",
            "Rendering_Job_Name": data_rendering_job_name,
            "Start_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
            "Render_Factor": render_factor,
            "Render_Resolution_Width": render_resolution_width,
            "Render_Resolution_Height": render_resolution_height,
            "Render_FPS": render_fps,
            "ZipNeRF_Gin_Config": zipnerf_gin_config,
            "CamP_Gin_Config": camp_gin_config,
            "GCS_Keyframes_File": gcs_keyframes_file,
            "GCS_Render_Path_File": gcs_render_path_file,
            "Render_Camtype": render_camtype,
            "Render_Path_Frames": render_path_frames,
            "GCS_Experiment_Path": output_colmap_dir_gcs,
            "Machine_Type": machine_type,
            "Accelerator_Type": accelerator_type,
            "Accelerator_Count": accelerator_count,
        }

        insert_or_update_row(training_row_data, training_table_id)
        insert_or_update_row(rendering_row_data, rendering_table_id)

        while True:
            try:
                if data_rendering_custom_job.resource_name:
                    rendering_job_id = data_rendering_custom_job.resource_name.split(
                        "/"
                    )[-1]
                    rendering_job_status = get_vertex_ai_job_status(rendering_job_id)
                    rendering_status_thread = threading.Thread(
                        target=update_job_info,
                        args=(
                            str(rendering_job_id),
                            rendering_table_id,
                            scene_name,
                            selected_folder,
                        ),
                    )
                    rendering_status_thread.start()
                    row_data = {
                        "Job_Status": JOB_STATE_MAPPING[int(rendering_job_status)],
                        "Experiment_ID": selected_folder,
                        "Scene_Name": scene_name,
                        "Image_Count": image_count,
                        "Colmap_Job_ID": colmap_job_id,
                        "Training_Job_ID": training_job_id,
                        "Training_Job_Name": training_job_name,
                        "Rendering_Job_ID": rendering_job_id,
                        "Rendering_Job_Name": data_rendering_job_name,
                        "GCS_Experiment_Path": output_colmap_dir_gcs,
                    }
                    insert_or_update_row(row_data, rendering_table_id)
                    break
            except RuntimeError:
                pass

        return get_bq_folders_dataframe_rendering(rendering_table_id)

    rendering_tip_text = """
        1. Upload images or videos in the Pose Estimation tab.
        2. Set parameters for rendering.
        3. Select dataset in the table.
        4. Click the **Run Rendering** button.
        5. After the rendering job starts, check the job status at
           [Vertex Custom Training](https://console.cloud.google.com/vertex-ai/training/custom-jobs?project={PROJECT_ID}).
        """

    def create_dropdown(label, options, default):
        return gr.Dropdown(options, label=label, interactive=True, value=default)

    def create_textbox(label, default, visible=True):
        return gr.Textbox(
            label=label, value=default, lines=1, interactive=True, visible=visible
        )

    def create_number(label, default):
        return gr.Number(label=label, value=default, interactive=True)

    def selected_video_output(selected_value):
        rendering_df = get_bq_folders_dataframe_rendering(rendering_table_id)
        selected_value = extract_dataset_id(selected_value)
        gcs_experiment_path = rendering_df[
            rendering_df["Experiment ID"] == selected_value
        ]["GCS Experiment Path"].iloc[0]
        rendering_job_name = rendering_df[
            rendering_df["Experiment ID"] == selected_value
        ]["Rendering Job Name"].iloc[0]
        rendering_job_status = rendering_df[
            rendering_df["Experiment ID"] == selected_value
        ]["Job Status"].iloc[0]
        experiment_path_no_prefix = gcs_experiment_path.replace("gs://", "")
        bucket_name = experiment_path_no_prefix.split("/")[0]
        folder_path = os.path.join(*experiment_path_no_prefix.split("/")[1:])
        if rendering_job_status == "SUCCEEDED":
            gcs_video_path = os.path.join(
                folder_path, "render", rendering_job_name, "path_videos", "videos"
            )
            try:
                color_video_path = list_mp4_files(
                    bucket_name, gcs_video_path, rendering_job_status
                )[1]
                remote_gcs_video_path = os.path.join(bucket_name, color_video_path)
                remote_gcs_video_path = "gs://" + remote_gcs_video_path
                download_gcs_file_to_local_dir(remote_gcs_video_path, "/tmp/")
                video_filename = os.path.basename(remote_gcs_video_path)
                temp_video_file_path = f"/tmp/{video_filename}"
            except IndexError:
                temp_video_file_path = create_pending_video(
                    rendering_job_status, "/tmp/pending.mp4"
                )
            return temp_video_file_path

    def on_row_select(rendering_df, evt: gr.SelectData):
        row_index = evt.index[0]
        if 0 <= row_index < len(rendering_df):
            selected_value = rendering_df.iloc[row_index, 0]
            rendering_job_status = rendering_df[
                rendering_df["Experiment ID"] == selected_value
            ]["Job Status"].iloc[0]
            (
                selected_video_output(selected_value)
                if rendering_job_status == "SUCCEEDED"
                else None
            )
            selected_video.visible = rendering_job_status == "SUCCEEDED"
            rendering_job_id = rendering_df[
                rendering_df["Experiment ID"] == selected_value
            ]["Rendering Job ID"].iloc[0]
            rendering_job_link = selected_value
            if rendering_job_status not in ["NOT STARTED"]:
                if rendering_job_id == "MANUAL":
                    pass
                elif "nerf-pipeline" in rendering_job_id:
                    rendering_job_link = get_vertex_ai_pipeline_run_link(
                        rendering_job_id, PROJECT_NUMBER, REGION
                    )
                    rendering_job_link = f"[{selected_value}]({rendering_job_link})"
                else:
                    rendering_job_link = get_vertex_ai_training_job_link(
                        rendering_job_id, PROJECT_NUMBER, REGION
                    )
                    rendering_job_link = f"[{selected_value}]({rendering_job_link})"
            return selected_value, rendering_job_link, gr.update(visible=True)
        else:
            return (
                gr.update(visible=False),
                gr.update(visible=False),
                gr.update(visible=False),
            )

    with gr.Blocks() as demo:
        with gr.Accordion("How To Use", open=False):
            gr.Markdown(rendering_tip_text)

        with gr.Accordion("Trained Checkpoints", open=True):
            rendering_dataframe = gr.Dataframe(
                value=get_bq_folders_dataframe_rendering(rendering_table_id),
                interactive=False,
            )
            run_rendering_button = gr.Button("RUN RENDERING", visible=False)
            selected_value = gr.Textbox(visible=False)
            selected_value_link = gr.Markdown()
            selected_video = gr.Interface(
                fn=selected_video_output,
                inputs=[selected_value],
                outputs="video",
                submit_btn="Play",
                visible=False,
            )
            rendering_dataframe.select(
                on_row_select,
                inputs=[rendering_dataframe],
                outputs=[selected_value, selected_value_link, run_rendering_button],
            )

        with gr.Row(equal_height=True):
            with gr.Column():
                gr.Markdown("### SET RENDERING PARAMETERS")
                gcs_keyframes_file = create_textbox("GCS Keyframe File", "")
                render_factor = create_dropdown("Downscaling Factor", [0, 2, 4, 8], 4)
                render_resolution_width = create_number("Video Resolution Width", 1280)
                render_resolution_height = create_number("Video Resolution Height", 720)
                render_fps = create_number("Video FPS", 30)
                machine_dropdown = create_dropdown(
                    "Select Machine Type",
                    ["NVIDIA_TESLA_V100", "NVIDIA_TESLA_A100"],
                    "NVIDIA_TESLA_V100",
                )
                with gr.Accordion("Advanced Settings", open=False):
                    zipnerf_gin_config = create_textbox(
                        "The ZipNeRF .gin Configuration File",
                        "configs/zipnerf/360_aglo128.gin",
                    )
                    camp_gin_config = create_textbox(
                        "The CamP .gin Configuration File",
                        "configs/camp/camera_optim.gin",
                    )
                    gcs_render_path_file = create_textbox(
                        "The GCS Render Path .npy File", ""
                    )
                    render_camtype = create_textbox(
                        "The Render Camera Type", "perspective"
                    )
                    render_path_frames = create_number(
                        "The Number of Frame along the Render Path", 120
                    )

            ws_table_id = create_textbox(
                "Workspace Table ID", rendering_table_id, visible=False
            )

        run_rendering_button.click(
            start_rendering,
            inputs=[
                selected_value,
                gcs_keyframes_file,
                render_factor,
                render_resolution_width,
                render_resolution_height,
                render_fps,
                zipnerf_gin_config,
                camp_gin_config,
                gcs_render_path_file,
                render_camtype,
                render_path_frames,
                machine_dropdown,
            ],
            outputs=[rendering_dataframe],
            show_progress=True,
            concurrency_limit=10,
        )

        demo.load(
            get_bq_folders_dataframe_rendering,
            inputs=[ws_table_id],
            outputs=[rendering_dataframe],
            concurrency_limit=10,
        )

        return demo, rendering_dataframe

In [42]:
# @title Pipeline Workshop


def create_pipeline_workshop():

    def get_worker_pool_specs(docker_uri, args, machine_type, accelerator_type):
        return [
            {
                "machine_spec": {
                    "machine_type": machine_type,
                    "accelerator_type": accelerator_type,
                    "accelerator_count": 8,
                },
                "replica_count": 1,
                "container_spec": {
                    "image_uri": docker_uri,
                    "args": args,
                },
            }
        ]

    def process_emails(email_input):
        # Split the input string by commas and strip any whitespace
        email_list = [email.strip() for email in email_input.split(",")]
        # You can add further validation here if needed
        return email_list

    def start_pipeline(
        selected_row,
        matcher_dropdown,
        camera_dropdown,
        video_frame_fps,
        max_num_features,
        mapper_dropdown,
        training_factor,
        training_max_steps,
        zipnerf_gin_config,
        camp_gin_config,
        gcs_keyframes_file,
        render_factor,
        render_resolution_width,
        render_resolution_height,
        render_fps,
        gcs_render_path_file,
        render_camtype,
        render_path_frames,
        machine_dropdown,
        email_list_input,
    ):

        print("Launching pipeline...")

        selected_row = extract_dataset_id(selected_row)
        folders_df = get_bq_folders_dataframe_colmap(colmap_table_id)

        if not email_list_input:
            gr.Warning("Please provide at least an email address.")
            return folders_df

        gcs_dataset_path_query = folders_df[
            folders_df["Experiment ID"] == selected_row
        ]["GCS Dataset Path"]
        gcs_dataset_path = (
            gcs_dataset_path_query.iloc[0] if not gcs_dataset_path_query.empty else None
        )

        input_image_folder = f"{BUCKET_NAME}/{selected_row}"
        output_dir = input_image_folder.replace("dataset", "experiment")
        data_calibration_job_name = get_job_name_with_datetime(
            "cloudnerf_gradio_colmap"
        )
        data_training_job_name = get_job_name_with_datetime("cloudnerf_gradio_training")
        data_rendering_job_name = get_job_name_with_datetime(
            "cloudnerf_gradio_rendering"
        )
        video_resolution = f"({render_resolution_width}, {render_resolution_height})"
        unique_experiments.add(selected_row)

        machine_type = (
            "n1-highmem-64"
            if machine_dropdown == "NVIDIA_TESLA_V100"
            else "a2-highgpu-8g"
        )
        accelerator_type = machine_dropdown
        accelerator_count = 8

        calibration_args = [
            "-use_gpu",
            "1",
            "-gcs_dataset_path",
            gcs_dataset_path,
            "-gcs_experiment_path",
            output_dir,
            "-camera",
            camera_dropdown,
            "-matching_strategy",
            MATCHER_MAPPING[matcher_dropdown],
            "-max_num_features",
            str(max_num_features),
            "-use_hierarchical_mapper",
            str(int(mapper_dropdown)),
            "-fps",
            str(video_frame_fps),
        ]

        training_args = [
            "-training_job_name",
            data_training_job_name,
            "-gcs_experiment_path",
            output_dir,
            "-factor",
            str(training_factor),
            "-max_steps",
            str(training_max_steps),
            "-gin_config_zipnerf",
            zipnerf_gin_config,
            "-gin_config_camp",
            camp_gin_config,
        ]

        rendering_args = [
            "-rendering_job_name",
            data_rendering_job_name,
            "-training_job_name",
            data_training_job_name,
            "-gcs_experiment_path",
            output_dir,
            "-render_resolution",
            video_resolution,
            "-render_video_fps",
            str(render_fps),
            "-factor",
            str(render_factor),
            "-gin_config_zipnerf",
            zipnerf_gin_config,
            "-gin_config_camp",
            camp_gin_config,
        ]

        if gcs_keyframes_file:
            rendering_args.append("-gcs_keyframes_file")
            rendering_args.append(gcs_keyframes_file)
        if render_camtype:
            rendering_args.append("-render_camtype")
            rendering_args.append(render_camtype)
        if gcs_render_path_file and not gcs_keyframes_file:
            rendering_args.append("-gcs_render_path_file")
            rendering_args.append(gcs_render_path_file)

        calibration_worker_pool_specs = get_worker_pool_specs(
            CALIBRATION_DOCKER_URI, calibration_args, machine_type, accelerator_type
        )
        training_worker_pool_specs = get_worker_pool_specs(
            TRAINING_DOCKER_URI, training_args, machine_type, accelerator_type
        )
        rendering_worker_pool_specs = get_worker_pool_specs(
            RENDERING_DOCKER_URI, rendering_args, machine_type, accelerator_type
        )

        scene_name = (
            folders_df[folders_df["Experiment ID"] == selected_row]["Scene Name"].iloc[
                0
            ]
            if not folders_df[folders_df["Experiment ID"] == selected_row][
                "Scene Name"
            ].empty
            else "NOT FOUND"
        )
        image_count = (
            int(
                folders_df[folders_df["Experiment ID"] == selected_row][
                    "Image Count"
                ].iloc[0]
            )
            if not folders_df[folders_df["Experiment ID"] == selected_row][
                "Image Count"
            ].empty
            else 0
        )

        email_list_processed = process_emails(email_list_input)
        nerf_pipeline_run = create_pipeline(
            data_calibration_job_name,
            data_training_job_name,
            data_rendering_job_name,
            calibration_worker_pool_specs,
            training_worker_pool_specs,
            rendering_worker_pool_specs,
            email_list_processed,
        )

        pipeline_job = run_pipeline(nerf_pipeline_run)

        while True:
            try:
                pipeline_job_id = pipeline_job.resource_name.split("/")[-1]
                for i, table_id in enumerate(
                    [colmap_table_id, training_table_id, rendering_table_id]
                ):
                    thread = threading.Thread(
                        target=update_pipeline_job_info,
                        args=(
                            str(pipeline_job_id),
                            table_id,
                            scene_name,
                            selected_row,
                            i + 1,
                        ),
                    )
                    thread.start()

                colmap_row_data = {
                    "Job_Status": pipeline_job.state,
                    "Pipeline_State": pipeline_job.state,
                    "Pipeline_Resource_Name": pipeline_job.resource_name,
                    "Experiment_ID": selected_row,
                    "Scene_Name": scene_name,
                    "Image_Count": image_count,
                    "Colmap_Job_ID": pipeline_job_id,
                    "Created_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
                    "Matcher_Type": MATCHER_MAPPING[matcher_dropdown],
                    "Camera_Type": camera_dropdown,
                    "Video_Frame_FPS": video_frame_fps,
                    "Max_Num_Features": max_num_features,
                    "Use_Hierarchical_Mapper": mapper_dropdown,
                    "GCS_Dataset_Path": gcs_dataset_path,
                    "GCS_Experiment_Path": output_dir,
                    "Machine_Type": machine_type,
                    "Accelerator_Type": accelerator_type,
                    "Accelerator_Count": accelerator_count,
                }

                training_row_data = {
                    "Job_Status": pipeline_job.state,
                    "Pipeline_State": pipeline_job.state,
                    "Pipeline_Resource_Name": pipeline_job.resource_name,
                    "Experiment_ID": selected_row,
                    "Scene_Name": scene_name,
                    "Image_Count": image_count,
                    "Colmap_Job_ID": pipeline_job_id,
                    "Training_Job_ID": pipeline_job_id,
                    "Training_Job_Name": data_training_job_name,
                    "Start_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
                    "Training_Factor": training_factor,
                    "Training_Max_Steps": training_max_steps,
                    "ZipNeRF_Gin_Config": zipnerf_gin_config,
                    "CamP_Gin_Config": camp_gin_config,
                    "GCS_Experiment_Path": output_dir,
                    "Machine_Type": machine_type,
                    "Accelerator_Type": accelerator_type,
                    "Accelerator_Count": accelerator_count,
                }

                rendering_row_data = {
                    "Job_Status": pipeline_job.state,
                    "Pipeline_State": pipeline_job.state,
                    "Pipeline_Resource_Name": pipeline_job.resource_name,
                    "Experiment_ID": selected_row,
                    "Scene_Name": scene_name,
                    "Image_Count": image_count,
                    "Colmap_Job_ID": pipeline_job_id,
                    "Training_Job_ID": pipeline_job_id,
                    "Training_Job_Name": data_training_job_name,
                    "Rendering_Job_ID": pipeline_job_id,
                    "Rendering_Job_Name": data_rendering_job_name,
                    "Start_Time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
                    "Render_Factor": render_factor,
                    "Render_Resolution_Width": render_resolution_width,
                    "Render_Resolution_Height": render_resolution_height,
                    "Render_FPS": render_fps,
                    "ZipNeRF_Gin_Config": zipnerf_gin_config,
                    "CamP_Gin_Config": camp_gin_config,
                    "GCS_Keyframes_File": gcs_keyframes_file,
                    "GCS_Render_Path_File": gcs_render_path_file,
                    "Render_Camtype": render_camtype,
                    "Render_Path_Frames": render_path_frames,
                    "GCS_Experiment_Path": output_dir,
                    "Machine_Type": machine_type,
                    "Accelerator_Type": accelerator_type,
                    "Accelerator_Count": accelerator_count,
                }
                insert_or_update_row(colmap_row_data, colmap_table_id)
                insert_or_update_row(training_row_data, training_table_id)
                insert_or_update_row(rendering_row_data, rendering_table_id)
                break

            except RuntimeError as e:
                print(e)

        folders_df = get_bq_folders_dataframe_colmap(colmap_table_id)
        return folders_df

    camp_zipnerf_pipeline_tip_text = """
        1. Upload images or videos in the Pose Estimation tab.
        2. Set parameters for the pipeline.
        3. Select dataset in the table.
        4. Click the **Run Full Pipeline** button.
        5. After the training job starts, check the pipeline jobs status at
           [Vertex Custom Training](https://console.cloud.google.com/vertex-ai/training/custom-jobs?project={PROJECT_ID}).
        """

    with gr.Blocks() as demo:

        def on_row_select(folders_df, evt: gr.SelectData):
            row_index = evt.index[0]
            if 0 <= row_index < len(folders_df):
                selected_value = folders_df.iloc[row_index, 0]
                colmap_job_id = folders_df[
                    folders_df["Experiment ID"] == selected_value
                ]["Colmap Job ID"].iloc[0]
                job_status = folders_df[folders_df["Experiment ID"] == selected_value][
                    "Job Status"
                ].iloc[0]
                colmap_job_link = selected_value
                if job_status not in ["NOT STARTED"]:
                    if "nerf-pipeline" in colmap_job_id:
                        colmap_job_link = get_vertex_ai_pipeline_run_link(
                            colmap_job_id, PROJECT_NUMBER, REGION
                        )
                        colmap_job_link = f"[{selected_value}]({colmap_job_link})"
                    elif colmap_job_id == "MANUAL":
                        pass
                    else:
                        colmap_job_link = get_vertex_ai_training_job_link(
                            colmap_job_id, PROJECT_NUMBER, REGION
                        )
                        colmap_job_link = f"[{selected_value}]({colmap_job_link})"
                return colmap_job_link, gr.update(visible=True), gr.update(visible=True)
            else:
                return "", gr.update(visible=False), gr.update(visible=True)

        with gr.Accordion("How To Use", open=False):
            gr.Markdown(camp_zipnerf_pipeline_tip_text)

        with gr.Accordion("Datasets", open=True):
            folders_dataframe = gr.Dataframe(
                value=get_bq_folders_dataframe_colmap(colmap_table_id),
                interactive=False,
            )
            selected_row = gr.Markdown(label="Selected Experiment", value="")
            run_pipeline_button = gr.Button("RUN FULL PIPELINE", visible=False)
            folders_dataframe.select(
                on_row_select,
                inputs=[folders_dataframe],
                outputs=[selected_row, run_pipeline_button],
            )

        def create_dropdown(label, options, default):
            return gr.Dropdown(options, label=label, interactive=True, value=default)

        def create_textbox(label, default, visible=True):
            return gr.Textbox(
                label=label, value=default, lines=1, interactive=True, visible=visible
            )

        def create_number(label, default):
            return gr.Number(label=label, value=default, interactive=True)

        gr.Markdown("### SET PIPELINE PARAMETERS")
        with gr.Accordion("Set COLMAP Parameters", open=False):
            with gr.Row(equal_height=True):
                with gr.Column():
                    matcher_dropdown = create_dropdown(
                        "Choose a Matching Algorithm",
                        [
                            "Exhaustive Matcher",
                            "Sequential Matcher",
                            "Spatial Matcher",
                            "Transitive Matcher",
                            "Vocab Tree Matcher",
                        ],
                        "Exhaustive Matcher",
                    )
                    camera_dropdown = create_dropdown(
                        "Type of Camera Used for Capture",
                        ["OPENCV", "OPENCV_FISHEYE"],
                        "OPENCV",
                    )
                    with gr.Accordion("Advanced Settings", open=False):
                        video_frame_fps = create_number("Video Frame Extraction FPS", 4)
                        max_num_features = create_number(
                            "Maximum Number Length of SIFT Descriptor", 8192
                        )
                        mapper_dropdown = create_dropdown(
                            "Use Hierarchical Mapper", [False, True], False
                        )

        with gr.Accordion("Set Training Parameters", open=False):
            with gr.Row(equal_height=True):
                with gr.Column():
                    training_factor = create_dropdown(
                        "Downscaling Factor", [0, 2, 4, 8], 4
                    )
                    training_max_steps = create_number(
                        "Maximum Number of Training Steps", 25000
                    )
                    with gr.Accordion("Advanced Settings", open=False):
                        zipnerf_gin_config = create_textbox(
                            "The ZipNeRF .gin Configuration File",
                            "configs/zipnerf/360_aglo128.gin",
                        )
                        camp_gin_config = create_textbox(
                            "The CamP .gin Configuration File",
                            "configs/camp/camera_optim.gin",
                        )

        with gr.Accordion("Set Rendering Parameters", open=False):
            with gr.Row(equal_height=True):
                with gr.Column():
                    gcs_keyframes_file = create_textbox("GCS Keyframe File", "")
                    render_factor = create_dropdown(
                        "Downscaling Factor", [0, 2, 4, 8], 4
                    )
                    render_resolution_width = create_number(
                        "Video Resolution Width", 1280
                    )
                    render_resolution_height = create_number(
                        "Video Resolution Height", 720
                    )
                    render_fps = create_number("Video FPS", 30)
                    machine_dropdown = create_dropdown(
                        "Select Machine Type",
                        ["NVIDIA_TESLA_V100", "NVIDIA_TESLA_A100"],
                        "NVIDIA_TESLA_V100",
                    )
                    with gr.Accordion("Advanced Settings", open=False):
                        gcs_render_path_file = create_textbox(
                            "The GCS Render Path .npy File", ""
                        )
                        render_camtype = create_textbox(
                            "The Render Camera Type", "perspective"
                        )
                        render_path_frames = create_number(
                            "The Number of Frame along the Render Path", 120
                        )

        with gr.Row(equal_height=True):
            _ = gr.Textbox(label="", interactive=False, visible=False)
            email_input = gr.Textbox(
                label="Set Notification Emails",
                placeholder="example1@example.com, example2@example.com",
            )

        ws_table_id = create_textbox(
            "Workspace Table ID", colmap_table_id, visible=False
        )

        run_pipeline_button.click(
            start_pipeline,
            inputs=[
                selected_row,
                matcher_dropdown,
                camera_dropdown,
                video_frame_fps,
                max_num_features,
                mapper_dropdown,
                training_factor,
                training_max_steps,
                zipnerf_gin_config,
                camp_gin_config,
                gcs_keyframes_file,
                render_factor,
                render_resolution_width,
                render_resolution_height,
                render_fps,
                gcs_render_path_file,
                render_camtype,
                render_path_frames,
                machine_dropdown,
                email_input,
            ],
            outputs=[folders_dataframe],
            show_progress=True,
            concurrency_limit=10,
        )

        demo.load(
            get_bq_folders_dataframe_colmap,
            inputs=[ws_table_id],
            outputs=[folders_dataframe],
            concurrency_limit=10,
        )
        return demo, folders_dataframe

In [ ]:
# @title Application Main


css = """
    .gradio-container {
      width: 90% !important;
      color: #000 !important;
    }
"""

with gr.Blocks(
    css=css, theme=gr.themes.Default(primary_hue="orange", secondary_hue="blue")
) as demo:
    gr.Markdown("# Model Garden Playground for CamP ZipNeRF")

    ws_colmap_table_id = gr.Textbox(
        value=colmap_table_id,
        lines=1,
        interactive=False,
        visible=False,
    )
    ws_training_table_id = gr.Textbox(
        value=training_table_id,
        lines=1,
        interactive=False,
        visible=False,
    )
    ws_rendering_table_id = gr.Textbox(
        value=rendering_table_id,
        lines=1,
        interactive=False,
        visible=False,
    )

    with gr.Tabs():
        with gr.TabItem("Pose Estimation") as tab1:
            pose_estimation_workshop, folders_dataframe_colmap = (
                create_pose_estimation_workshop()
            )
            gr.on(
                [tab1.select],
                get_bq_folders_dataframe_colmap,
                inputs=[ws_colmap_table_id],
                outputs=[folders_dataframe_colmap],
            )
        with gr.TabItem("Training") as tab2:
            training_workshop, folders_dataframe_training = create_training_workshop()
            gr.on(
                [tab2.select],
                get_bq_folders_dataframe_training,
                inputs=[ws_training_table_id],
                outputs=[folders_dataframe_training],
            )
        with gr.TabItem("Rendering") as tab3:
            rendering_workshop, folders_dataframe_rendering = (
                create_rendering_workshop()
            )
            gr.on(
                [tab3.select],
                get_bq_folders_dataframe_rendering,
                inputs=[ws_rendering_table_id],
                outputs=[folders_dataframe_rendering],
            )
        with gr.TabItem("Pipeline") as tab4:
            pipeline_workshop, folders_dataframe_pipeline = create_pipeline_workshop()
            gr.on(
                [tab4.select],
                get_bq_folders_dataframe_colmap,
                inputs=[ws_colmap_table_id],
                outputs=[folders_dataframe_pipeline],
            )


show_debug_logs = True
demo.queue(max_size=10)
demo.launch(
    share=True, inline=False, debug=show_debug_logs, show_error=True, max_threads=10
)

INFO:google.cloud.aiplatform.jobs:CustomJob projects/816369962409/locations/us-central1/customJobs/3969543499497340928 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/816369962409/locations/us-central1/customJobs/3969543499497340928 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/816369962409/locations/us-central1/customJobs/3969543499497340928 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/816369962409/locations/us-central1/customJobs/3969543499497340928 current state:
JobState.JOB_STATE_PENDING
